In [1]:
import torch
import numpy as np


In [2]:
n_train, n_test, true_w, true_b = 100, 100, [1.2, -3.4, 5.6], 5
features = torch.randn((n_train + n_test, 1))
poly_features = torch.cat(
    (features, torch.pow(features, 2), torch.pow(features, 3)), 1)
labels = (true_w[0] * poly_features[:, 0] + true_w[1] * poly_features[:, 1] +
          true_w[2] * poly_features[:, 2] + true_b)
labels += torch.tensor(
    np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float32)

In [3]:
features[:2],poly_features[:2],labels[:2]

(tensor([[-0.1988],
         [-0.2163]]), tensor([[-0.1988,  0.0395, -0.0079],
         [-0.2163,  0.0468, -0.0101]]), tensor([4.5769, 4.5262]))

In [4]:
num_epochs,loss=100,torch.nn.MSELoss()

In [5]:
import torchvision
import matplotlib.pyplot as plt
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [6]:
def fit_and_plot(train_features,test_features,train_labels,test_labels):
    net=torch.nn.Linear(train_features.shape[-1],1)
    batch_size=min(10,train_labels.shape[0])
    dataset=torch.utils.data.TensorDataset(train_features,train_labels)
    train_iter=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True)
    optimizer=torch.optim.SGD(net.parameters(),lr=0.01)
    train_ls,test_ls=[],[]
    for _ in range(num_epochs):
        for x,y in train_iter:
            l=loss(net(x),y.view(-1,1))
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
        train_labels=train_labels.view(-1,1)
        test_labels=test_labels.view(-1,1)
        train_ls.append(loss(net(train_features),train_labels).item())
        test_ls.append(loss(net(test_features),test_labels).item())
    print('final epoch : train loss ',train_ls[-1],'test loss : ' ,test_ls[-1])
    print('weight :',net.weight.data,' bias :',net.bias.data)
    plt.plot(range(1,num_epochs+1),train_ls,'b-')
    plt.plot(range(1,num_epochs+1),test_ls,'r--')
    plt.show()
#     for p in net.parameters():
#         print(p)
    
    

In [7]:
fit_and_plot(poly_features[:n_train, :], poly_features[n_train:, :],
labels[:n_train], labels[n_train:])

final epoch : train loss  0.00011758734763134271 test loss :  0.00016712353681214154
weight : tensor([[ 1.2098, -3.3981,  5.5978]])  bias : tensor([4.9967])


In [8]:
fit_and_plot(features[:n_train,:],features[n_train:,:],labels[:n_train],
            labels[n_train:])

final epoch : train loss  81.84788513183594 test loss :  182.2057647705078
weight : tensor([[14.4040]])  bias : tensor([3.0819])
